### Edgar Moises Hernandez-Gonzalez
#### 25/06/20
#### Clasificacion de calculo mental en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Resize proporcional de (45 x n_canales / 2) x 1440 / 2 [720x672]

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import time
import pywt
import cv2
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
# todos los canales
x = pd.read_csv("/content/drive/My Drive/MentalCalculation/Datos/MeCa_car_01_3.csv", header=None)

# etiquetas
y = pd.read_csv("/content/drive/My Drive/MentalCalculation/Datos/EtiquetasMeCa01ACLL-03.csv", header=None).values

In [ ]:
print(x.shape)
print(y.shape)

(304, 43008)
(304, 1)


In [ ]:
# unir 32 canales y resize de tamaño proporcional
def unir_escalogramas_vertical(data):
  dim = (672, 720) #ancho, alto
  fs = 512
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],720,672))
  temporal = np.zeros((1440,1344))
  for i in range(data.shape[0]): #n muestras
    for j in range(32): #n canales
      sig = data.iloc[i, j*1344:(j+1)*1344]
      #sig = sig[160:1184]
      #escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / 512))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [ ]:
# llamar a unir_escalogramas_vertical
inicio = time.time()

x = unir_escalogramas_vertical(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
Tiempo: 454.87087535858154


In [ ]:
print(x.shape)

(304, 720, 672)


In [ ]:
# convertir a float
x = x.astype('float32')

# escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [ ]:
# convertir de 3D a 4D
x = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))

print(x.shape)

(304, 720, 672, 1)


In [ ]:
model = load_model('/content/drive/My Drive/MentalCalculation/Modelos/Model_CWT_01_1_CV.hdf5')

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 720, 672, 8)       368       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 360, 336, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 360, 336, 8)       2888      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 180, 168, 8)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 241920)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                15482944  
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)               

In [ ]:
test_loss, test_acc = model.evaluate(x, y, verbose=0)

In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.6318
accuracy: 0.6151


In [ ]:
probabilidades = model.predict(x)
y_pred_todo = np.argmax(probabilidades, 1) #1=fila

In [ ]:
f1 = f1_score(y, y_pred_todo, average='macro')
print("f1:", f1)

f1: 0.6011080332409972


In [ ]:
matriz_confusion = confusion_matrix(y, y_pred_todo)
print("confusion matrix:\n", matriz_confusion)  

confusion matrix:
 [[122   3]
 [114  65]]


In [ ]:
probabilidades = probabilidades[:, 1]
auc = roc_auc_score(y, probabilidades)
#fpr, tpr, thresholds = roc_curve(y, probabilidades)

print("AUC: %.4f" % auc)
#plt.plot(fpr, tpr)
#plt.grid()
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.show()

AUC: 0.8498
